In [14]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors

In [15]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)

In [16]:
from pinocchio.visualize import GepettoVisualizer

### HYPER PARAMETERS
# Hyperparameters defining the optimal control problem.
T = 10
DT = 0.05

### LOAD AND DISPLAY SOLO
# Load the robot model from example robot data and display it if possible in Gepetto-viewer
viz_debug = GepettoVisualizer(robot.model,robot.collision_model,robot.visual_model)
viz_debug.initViewer()
viz_debug.loadViewerModel()

viz_debug.display(robot.q0)

In [3]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
viz.display(robot.q0)

In [12]:
class KineModel:
    def __init__ (self, cmodel):
        self.cmodel = cmodel
        self.cdata = cdata = cmodel.createData()
        self.nq, self.nv = cmodel.nq, cmodel.nv
        self.nx = self.nq + self.nv
        self.ndx = 2* self.nv
        nq = self.nq,
        nv = self.nv
        nx = self.nx
        ndx = self.ndx

        cx = casadi.SX.sym('cx', nq+nv, 1)
        cdx = casadi.SX.sym('cx', 2* nv, 1)
        ca = casadi.SX.sym('ca', nv, 1)


        self.IDX_LF = IDX_LF = cmodel.getFrameId('leg_left_6_link')
        self.IDX_RF = IDX_RF = cmodel.getFrameId('leg_right_6_link')

        self.integrate = casadi.Function('integrate', [cx, cdx], [cpin.integrate(cmodel, cx[: nq], cdx[: nv])] )

        #self.com_position = casadi.Function('com', [cx], [cpin.centerOfMass(cmodel, cdata, cx[: nq])] )

        cpin.forwardKinematics(cmodel, cdata, cx[: nq], cx[nq :], ca)
        cpin.updateFramePlacements(cmodel, cdata)        
        self.lf_position = casadi.Function('lf', [cq], [cdata.oMf[IDX_LF].translation])
        self.rf_position = casadi.Function('rf', [cq], [cdata.oMf[IDX_RF].translation])
        self.rf_rotation = casadi.Function('rf', [cq], [cdata.oMf[IDX_RF].rotation])
        self.get_T_rf = casadi.Function('T_rf', [cq], [cpin.log6(cpin.SE3(self.rf_rotation(cq), self.rf_position(cq) ) ).vector])

    def cost(self, cx):
        com_pos = self.com_position(cx[: self.nq])
        return com_pos.T @ com_pos 

    def foot_constraint(self, cx, opti):
        opti.subject_to(self.get_T_rf(cx[: self.nq]) == 0)

        

In [13]:
kine_model = KineModel(cmodel)
kine_model.cx[: kine_model.nq].shape

TypeError: can only concatenate tuple (not "int") to tuple

In [11]:
nq, nv = cmodel.nq, cmodel.nv
cx = casadi.SX.sym('cx', nq+nv, 1)
cdx = casadi.SX.sym('cx', 2* nv, 1)
ca = casadi.SX.sym('ca', nv, 1)

#casadi.sumsqr(kine_model.foot_constraint(cq))

F = casadi.Function('integrate', [cx, cdx], [cpin.integrate(cmodel, cx[: nq], cdx[: nv])] )
F(cx, cdx)


SX(@1=4.93038e-32, @2=((sq(cx_3)+(sq(cx_4)+sq(cx_5)))+@1), @3=sqrt(@2), @4=0.00012207, @5=(@3<@4), @6=0.5, @7=24, @8=1, @9=((@5?(@6-(@2/@7)):0)+((!@5)?((@8-cos(@3))/@2):0)), @10=(@3<@4), @11=120, @12=((@10?(0.166667-(@2/@11)):0)+((!@10)?(((@3-sin(@3))/@2)/@3):0)), @13=((cx_3*cx_1)-(cx_4*cx_0)), @14=((cx_5*cx_0)-(cx_3*cx_2)), @15=((cx_0+(@9*((cx_4*cx_2)-(cx_5*cx_1))))+(@12*((cx_4*@13)-(cx_5*@14)))), @16=((cx_4*cx_2)-(cx_5*cx_1)), @17=((cx_2+(@9*((cx_3*cx_1)-(cx_4*cx_0))))+(@12*((cx_3*@14)-(cx_4*@16)))), @18=((cx_1+(@9*((cx_5*cx_0)-(cx_3*cx_2))))+(@12*((cx_5*@16)-(cx_3*@13)))), @19=((cx_4*@17)-(cx_5*@18)), @20=(@19+@19), @21=((cx_3*@18)-(cx_4*@15)), @22=(@21+@21), @23=((cx_5*@15)-(cx_3*@17)), @24=(@23+@23), @25=(sq(cx_3)+(sq(cx_4)+sq(cx_5))), @26=(@4<@25), @27=sqrt((@25+@1)), @28=(@6*@27), @29=sin(@28), @30=(@25/4), @31=(@6*((@8-(@30/6))+(sq(@30)/@11))), @32=((@26?(@29*(cx_3/@27)):0)+((!@26)?(@31*cx_3):0)), @33=(@4<@25), @34=2, @35=((@33?cos(@28):0)+((!@33)?((@8-(@30/@34))+(sq(@30)/@7)):

In [45]:
cq = casadi.MX.sym('cq', robot.nq, 1)
cv = casadi.MX.sym('cv', robot.nv, 1)
ca = casadi.MX.sym('ca', robot.nv, 1)


Function(T_rf:(i0[39])->(o0[6]) SXFunction)

In [29]:
ballId = 'world/ball'
viz.display(q0)
viz.addSphere(ballId, 0.1, color=colors.red)
pose = pin.SE3(kine_model.rf_rotation(q0).full(), kine_model.rf_position(q0).full() )
viz.applyConfiguration(ballId, pose)

In [65]:
opti = casadi.Opti()

dxs = opti.variable(kine_model.nx, 1)

opti.minimize(kine_model.cost(dxs[: kine_model.nq]))

opti.subject_to(casadi.sumsqr(dxs[3:7]) == 1) 
kine_model.foot_constraint(dxs[:kine_model.nq], opti)


In [66]:
opti.solver('ipopt')

opti.solve()

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       73
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      777

Total number of variables............................:       77
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

CasADi - 2022-03-23 09:40:48 WARNING("solver:nlp_hess_l failed: NaN detected for output hess_gamma_x_x, at nonzero index 6 (row 3, col 3).") [.../casadi/core/oracle_function.cpp:265]


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:997: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'

In [67]:
x_sol = opti.value(dxs)
q_sol = x_sol[: kine_model.nq]
v_sol = x_sol[kine_model.nq:]

In [64]:
viz.display(q_sol)